In [1]:
import pandas as pd
import os
from typing import Optional

def fusionner_csv_par_colonnes_robustes(
    fichier_gauche_path: str,
    fichier_droit_path: str,
    colonne_cle: str,
    type_jointure: str = 'inner'
) -> Optional[pd.DataFrame]:
    """
    Fusionne deux fichiers CSV par une colonne clé avec gestion d'erreurs.
    Vérifie la présence des fichiers, l'existence de la clé, et le format de la clé.
    
    Args:
        fichier_gauche_path: Chemin du premier fichier CSV.
        fichier_droit_path: Chemin du second fichier CSV.
        colonne_cle: Nom de la colonne à utiliser pour la jointure (ex: 'time').
        type_jointure: Type de jointure ('inner', 'left', 'right', 'outer').
                       
    Returns:
        Le DataFrame fusionné ou None en cas d'erreur critique.
    """
    
    # Paramètres de lecture basés sur votre format habituel
    params_lecture = {'sep': ';', 'decimal': ',', 'parse_dates': [colonne_cle]}
    
    # ------------------ 1. Gestion des erreurs de lecture/format ------------------
    try:
        if not os.path.exists(fichier_gauche_path) or not os.path.exists(fichier_droit_path):
            print("ERREUR CRITIQUE : Un ou plusieurs chemins de fichiers sont introuvables.")
            return None
            
        df_gauche = pd.read_csv(fichier_gauche_path, **params_lecture)
        df_droit = pd.read_csv(fichier_droit_path, **params_lecture)
        
    except Exception as e:
        print(f"ERREUR CRITIQUE lors de la lecture des fichiers (vérifiez le chemin ou les paramètres sep/decimal) : {e}")
        return None

    # ------------------ 2. Gestion des erreurs de colonne clé ------------------
    if colonne_cle not in df_gauche.columns:
        print(f"ERREUR : La colonne clé '{colonne_cle}' est absente du fichier de gauche.")
        return None
    if colonne_cle not in df_droit.columns:
        print(f"ERREUR : La colonne clé '{colonne_cle}' est absente du fichier de droite.")
        return None
        
    # ------------------ 3. Gestion des erreurs de format de clé (Datetime) ------------------
    # On vérifie que la colonne clé est de type datetime (après le parse_dates)
    if not pd.api.types.is_datetime64_any_dtype(df_gauche[colonne_cle]):
        print(f"ATTENTION : La colonne clé '{colonne_cle}' du fichier de gauche n'est pas au format Datetime.")
        # Tentative de conversion si la lecture a échoué silencieusement
        try:
            df_gauche[colonne_cle] = pd.to_datetime(df_gauche[colonne_cle])
        except:
            print(f"ERREUR : Impossible de convertir la clé '{colonne_cle}' du fichier de gauche en format Date/Heure.")
            return None

    if not pd.api.types.is_datetime64_any_dtype(df_droit[colonne_cle]):
        print(f"ATTENTION : La colonne clé '{colonne_cle}' du fichier de droite n'est pas au format Datetime.")
        try:
            df_droit[colonne_cle] = pd.to_datetime(df_droit[colonne_cle])
        except:
            print(f"ERREUR : Impossible de convertir la clé '{colonne_cle}' du fichier de droite en format Date/Heure.")
            return None

    # ------------------ 4. Fusion des DataFrames ------------------
    try:
        df_fusionne = pd.merge(
            df_gauche,
            df_droit,
            on=colonne_cle,
            how=type_jointure,
            suffixes=('_gauche', '_droit')
        )
        print(f"Fusion réussie avec la jointure '{type_jointure}' sur la clé '{colonne_cle}'.")
        return df_fusionne
        
    except Exception as e:
        print(f"ERREUR : Échec de l'opération pd.merge : {e}")
        return None

In [6]:
fusionner_csv_par_colonnes_robustes(
    "df_162_clean.csv",
    "csv_meteo.csv",
    "datetime",
    'inner'
)

ERREUR CRITIQUE lors de la lecture des fichiers (vérifiez le chemin ou les paramètres sep/decimal) : Missing column provided to 'parse_dates': 'datetime'


In [9]:
import pandas as pd
import os

# --- Définition des Chemins et Clé ---
fichier_meteo_path = 'csv_meteo.csv' # Le fichier avec la colonne 'time' (ou 'datetime' si renommé)
fichier_clean_path = 'df_162_clean.csv'  # Le fichier avec la colonne 'datetime' (assumée)
COLONNE_CLE = 'datetime'

# Paramètres de lecture communs (basés sur votre fichier météo)
# NOTE : Ces paramètres doivent être vérifiés et ajustés pour df_162_clean.csv si son format est différent.
PARAMS_LECTURE_METEO = {'sep': ';', 'decimal': ','}

# --- ÉTAPE 1 : Chargement et Harmonisation des Noms ---

# Chargement du fichier 1 (ma_meteo_export.csv)
try:
    df_meteo = pd.read_csv(fichier_meteo_path, **PARAMS_LECTURE_METEO)
    # Si l'utilisateur a confirmé avoir renommé 'time' en 'datetime' ailleurs, 
    # nous vérifions et renommons ici au cas où.
    if 'time' in df_meteo.columns:
        df_meteo.rename(columns={'time': COLONNE_CLE}, inplace=True)
    
    # Conversion explicite de la colonne clé en Datetime
    # df_meteo[COLONNE_CLE] = pd.to_datetime(df_meteo[COLONNE_CLE])
    print(f"1. Fichier {fichier_meteo_path} chargé et clé '{COLONNE_CLE}' prête.")

except Exception as e:
    print(f"ERREUR lors du chargement ou de la préparation de {fichier_meteo_path} : {e}")
    # Si le fichier n'a ni 'time' ni 'datetime', l'opération échouera ici.
    raise

# Chargement du fichier 2 (df_162_clean.csv)
try:
    # Lecture, en supposant que df_162_clean utilise aussi le séparateur ';' et décimal ','
    df_clean = pd.read_csv(fichier_clean_path, **PARAMS_LECTURE_METEO)
    
    # Conversion explicite de la colonne clé en Datetime
    # Cela gère l'hypothèse où 'datetime' existe, mais n'est pas reconnu comme datetime par défaut.
    # df_clean[COLONNE_CLE] = pd.to_datetime(df_clean[COLONNE_CLE])
    print(f"2. Fichier {fichier_clean_path} chargé et clé '{COLONNE_CLE}' prête.")

except Exception as e:
    print(f"ERREUR lors du chargement ou de la préparation de {fichier_clean_path}. Vérifiez le chemin, le séparateur (sep=';') et le décimal (decimal=',') : {e}")
    # Si la colonne 'datetime' n'existe toujours pas ou si le format de lecture est incorrect, l'erreur est là.
    raise

# --- ÉTAPE 2 : Fusion (Jointure Interne) ---
try:
    df_fusionne = pd.merge(
        df_clean,
        df_meteo,
        on=COLONNE_CLE,
        how='inner',
        suffixes=('_clean', '_meteo')
    )
    
    print(f"\n3. Fusion réussie. Jointure '{'inner'}' sur la clé '{COLONNE_CLE}'.")
    print(f"Taille du DataFrame final : {len(df_fusionne)} lignes.")
    print("\nAperçu du DataFrame fusionné :")
    print(df_fusionne.head())
    
except Exception as e:
    print(f"ERREUR lors de l'opération de fusion pd.merge : {e}")
    raise

1. Fichier csv_meteo.csv chargé et clé 'datetime' prête.
2. Fichier df_162_clean.csv chargé et clé 'datetime' prête.
ERREUR lors de l'opération de fusion pd.merge : 'datetime'


KeyError: 'datetime'